# Vibe-Hacking Tutorial "My first Hack" 🤖💻 

## Hacking PortSwigger Labs with CAI Framework

## 1. Introduction 

We explore how to use the **CAI Python API** to attack PortSwigger Labs. 

We combine:
- CAI’s red-team agent 
- PortSwigger’s vulnerability lab environments 
- Large Language Models (LLMs) for autonomous reasoning and payload generation


## 2. Objectives 

- Learn how LLMs assist in crafting attacks on web applications
- Test vulnerabilities using PortSwigger Web Security Academy
- Automate techniques to identify and fix vulnerabilities using CAI
- Analyse and defend using LLMs' log interpretation
- Learn how to use the CAI Python API

### Installing  Python Requirements

In [ ]:
%%capture
!pip install cai-framework
!pip install pandas
!pip install selenium
!pip install python-dotenv
!pip install nest-asyncio

Because this is a Jupyter Notebook, we need to have Jupyter properly configured and running locally on our computer in order to follow this tutorial successfully.  

If you need more information on setting up Jupyter Notebook, please visit [this link](https://docs.jupyter.org/en/latest/install/notebook-classic.html).

## 3. Getting the Portswigger Labs Environment 


### 3.1. Importing Required Modules 

In [ ]:
#Portswigger web scraper bot
import utils.portswiggerbot as pb

#Other utilities
import nest_asyncio
import os 
from dotenv import load_dotenv
load_dotenv(override=True)

### 3.2. Configuration Setup

### 3.2.1 Setting Up the CAI `.env` File

CAI specifies in the documentation that it is necessary to set up an .env file in the same folder as this notebook. For setting up the PortSwigger environment, two additional variables have been added to the template in CAI’s  [.env.example](https://github.com/aliasrobotics/cai/blob/main/.env.example). So, we need to create an .env file with the following variables:

```
PORTSWIGGER_USERNAME='your-portswigger-email'
PORTSWIGGER_PASSWORD='your-portswigger-password'
OPENAI_API_KEY='sk-123'
ANTHROPIC_API_KEY=''
OLLAMA=''
PROMPT_TOOLKIT_NO_CPR=1
CAI_STREAM=false
```
✅ Note on PortSwigger: If you don’t have a PortSwigger Labs account, you can create one [here](https://portswigger.net/users/register).

✅ Note on CAI: If you need more information on setting up the .env file, check out the [CAI documentation](https://github.com/aliasrobotics/cai/blob/main/README.md#nut_and_bolt-setup-env-file).


### 3.2.2 Setting Up the PortSwigger Bot


To extract the labs, we start by initialising the PortSwigger bot and loading the username and password from the .env file. Next, we specify the number of labs <b>(N_LABS)</b> to retrieve and the type of vulnerability we want to target <b>(SECTION)</b>. The supported vulnerability sections include:

- sql-injection
- cross-site-scripting
- cross-site-request-forgery-csrf
- clickjacking
- dom-based-vulnerabilities
- cross-origin-resource-sharing-cors
- xml-external-entity-xxe-injection
- server-side-request-forgery-ssrf
- http-request-smuggling
- os-command-injection
- server-side-template-injection
- path-traversal
- access-control-vulnerabilities
- authentication
- websockets
- web-cache-poisoning
- insecure-deserialization
- information-disclosure
- business-logic-vulnerabilities
- http-host-header-attacks
- oauth-authentication
- file-upload-vulnerabilities
- jwt
- essential-skills
- prototype-pollution
- graphql-api-vulnerabilities
- race-conditions
- nosql-injection
- api-testing
- web-llm-attacks
- web-cache-deception

In [ ]:
SECTION = "cross-site-scripting"
N_LABS = 3 # If you want to test all the labs in the section, change this to -1
USERNAME = os.getenv("PORTSWIGGER_USERNAME")
PASSWORD = os.getenv("PORTSWIGGER_PASSWORD")

### 3.3. Getting  PortSwigger labs
We can now extract the lab information by running the following line of code.

In [ ]:
#Initialize bot
bot = pb.Bot()

#Login to Portswigger Academy
bot.login(USERNAME,PASSWORD)

#Get labs urls by section or type of lab
topics = bot.choose_topic(SECTION)

#Get lab metadata
labs = [bot.obtain_lab_information(link) for link in topics[0:N_LABS]]

#format session cookies
cookies = bot.driver.get_cookies()

# Define names of essential cookies
essential_cookies = {'SessionId','Authenticated_UserVerificationId','t'}
# Extract only the essential cookies
essential_cookies = [cookie for cookie in cookies if cookie['name'] in essential_cookies]

## 4. Setting Up the CAI Red Team Agent


An Agent is an intelligent component that interacts with an environment. In cybersecurity, an Agent interacts with systems and networks, using peripherals and network interfaces as sensors, reasoning about the gathered information, and then executing network actions as if they were actuators.

In CAI, there are different types of predefined agents. You can take a look here:
-  [CAI Agents](https://github.com/aliasrobotics/cai/tree/main/src/cai/agents)

For this project, we will use the “Red Team Base Agent.” First, we need to specify the LLMs we want to use with it. We are going to use GTP 4.0.

In [ ]:
os.environ["CAI_MODEL"] = "openai/gpt-4o"

#### 4.1. Prompts explanation

CAI Agents work with two types of prompts:

##### System Prompt 
This prompt defines the overall context and instructs the LLMs to assume a specific role. In this case, the model’s goal is to identify and exploit vulnerabilities.

✅ Note: When using the built-in Red Team Agent from CAI, the system prompt is already configured for you, so there is no need to set it manually. You can find the default template for the agent here:
-  [Red Team Agent System Prompt](https://github.com/aliasrobotics/cai/blob/main/src/cai/prompts/system_red_team_agent.md)

##### User Prompt (customizable)

This prompt defines the specific task the agent should perform. For PortSwigger Labs, it should include:

- The lab title (placeholder: **LAB_TITLE**)
- The lab description (placeholder: **LAB_DESCRIPTION**)
- The lab environment URL (placeholder: **LAB_ENVIRONMENT_URL**)
- Session cookies to authenticate in the lab (placeholder: **COOKIES**)

✅ Note: We will use the following user prompt, so it is possible to modify it and interact with or create other types of prompts for LLMs.


In [1]:
prompt_example = f"""
attack **LAB_ENVIRONMENT_URL**
The title is **LAB_TITLE** 
The description is **LAB_DESCRIPTION** 

Please consider the following mandatory rules:
1. You have to connect to the session with the cookies **COOKIES**
2. Do not stop until you try all the commands or exploits you can run and create
3. Write the explanation of what you did in an HTML format easy to understand
4. Write every payload command and the exploits you create in the format <payloadgen>HERE</payloadgen>
5. Do not create files with the payload that you generate because you have to solve the lab by yourself
"""

Now that we have explained the agent and prompts, it is time to run our first AI-powered task, but first we must choose the target lab. and format the final user prompt with the data of the lab. In this case, we are going to use the following one:

In [ ]:
lab = labs[0]
lab

We can take a look at the formatted user prompt.

In [ ]:
user_prompt = (
    prompt_example.replace("**LAB_ENVIRONMENT_URL**", lab['environment_url'])
    .replace("**LAB_TITLE**", lab['title'])
    .replace("**LAB_DESCRIPTION**", lab['description'])
    .replace("**COOKIES**", str(essential_cookies))
           )
print(user_prompt)

## 5. Attacking PortSwigger Labs with CAI Red Team Agent


It’s time to have some fun hacking PortSwigger Labs! In the code below, we’ve created a helper function that runs the agent asynchronously. Just call the function, and your agent will automatically interact with the target data, searching for vulnerabilities and trying to exploit them

In [ ]:
# Import after environment setup
from cai.sdk.agents import Runner
from cai.agents.red_teamer import redteam_agent

# Allow async in jupyter notebooks
nest_asyncio.apply()


#function to run the agent asynchronously
async def run_agent(message):
    response = await Runner.run(redteam_agent, message)
    return response


In [ ]:
response = await run_agent(message=user_prompt)

## 6. Result Analysis

Now that our CAI agent has done its work, let’s break down the results to understand what it found, how it exploited the target, and what the structure of the final response is. In the following code, we can see the response type and how the output is displayed.

In [ ]:
print(f"The tipe of the response is a: {type(response)}")
response

### 6.1 Understanding the `RunResult`

When we run the CAI Red Team Agent, we get back a RunResult object.  
Think of it as a detailed report of  everything the agent did  during its hacking session.  
Here’s how to read it:


#### `input`

This is the user prompt we gave to the agent.  


####  `new_items`
This shows what the agent  produced during the run. This considers:

##### `MessageOutputItem`
Contains the following information:
  - The agent info  (Red Team Agent)
  - The  tools it used (like generic_linux_command to run shell commands, or execute_code to write and run exploits)
  - The output, which is a clear HTML report  explaining the methodology used to solve the lab.


#### `raw_responses`

This is the  raw output  from the agent, showing exactly what text the LLM produced.  


#### `final_output`

This is the  final, cleaned-up version of the report. 

In our example, it’s a complete HTML file  that explains:

- What the attack did
- The payload used:
  ```html
  <payloadgen><script>alert('XSS Lab')</script></payloadgen>
  ```
- How to inject it into the lab’s search parameter
- The result and security impact

## 7. Conclusion

In this notebook, we learned how to run a complete attack on the PortSwigger vulnerable lab using the CAI Python API.  

Now you are able to:
- Launch a CAI **Red Team Agent** 
- Provide clear instructions, cookies, and payload rules
- Let the agent autonomously craft and test exploits
- Review the results step by step in a structured `RunResult`

This shows the power of combining **LLM-driven reasoning** with real hacking tools.  
You can now adapt this approach to test other labs, real applications, or integrate it into your own red teaming workflows.

Next, try:
- Experimenting with different lab challenges on **PortSwigger Web Security Academy**
- Tuning your agent prompts for more advanced payloads
- Analysing the results to write better detection and defence rules
Happy hacking!

# Happy hacking! 🕵️‍♂️💻🔍  